#  Load taxi data and pre_handel

In [1]:
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import copy
import scipy.integrate as spi
import matplotlib.pyplot as plt
import pandas as pd
import geopandas as gpd

In [2]:
# load taxi travel data
data_taxi_fhv=pd.read_csv(r'C:\jupyter notebook\PAPER\data for taxi\fhv_tripdata_2020-03.csv')
data_taxi_fhvhv=pd.read_csv(r'C:\jupyter notebook\PAPER\data for taxi\fhvhv_tripdata_2020-03.csv')
data_taxi_green=pd.read_csv(r'C:\jupyter notebook\PAPER\data for taxi\green_tripdata_2020-03.csv')
data_taxi_yellow=pd.read_csv(r'C:\jupyter notebook\PAPER\data for taxi\yellow_tripdata_2020-03.csv')

data_taxi=pd.concat((data_taxi_fhv[['PULocationID','DOLocationID']],data_taxi_fhvhv[['PULocationID','DOLocationID']],data_taxi_green[['PULocationID','DOLocationID']],data_taxi_yellow[['PULocationID','DOLocationID']]),axis=0)
data_taxi=data_taxi.dropna().astype(int)
data_taxi['num']=1
data_taxi=data_taxi.reset_index(drop=True)
data_taxi 

,PULocationID,DOLocationID,num
0,264,264,1
1,264,264,1
2,264,264,1
3,264,264,1
4,264,264,1
...,...,...,...
17892296,137,87,1
17892297,137,71,1
17892298,137,32,1
17892299,137,159,1


In [3]:
# load taxi zone GIS data
data_zone=pd.read_csv(r'C:\jupyter notebook\PAPER\data for taxi\taxi zone look up table.csv')
data_zone

,LocationID,Borough,Zone,service_zone
0,1,EWR,Newark Airport,EWR
1,2,Queens,Jamaica Bay,Boro Zone
2,3,Bronx,Allerton/Pelham Gardens,Boro Zone
3,4,Manhattan,Alphabet City,Yellow Zone
4,5,Staten Island,Arden Heights,Boro Zone
...,...,...,...,...
260,261,Manhattan,World Trade Center,Yellow Zone
261,262,Manhattan,Yorkville East,Yellow Zone
262,263,Manhattan,Yorkville West,Yellow Zone
263,264,Unknown,NV,NaN


In [4]:
#data_taxi[data_taxi['PULocationID']==103]
#data_taxi[data_taxi['PULocationID']==104]
data_taxi[data_taxi['DOLocationID']==104]

,PULocationID,DOLocationID,num


##  It can be seen that there is no taxi orders in taxi zone 103,104. And taxi zone 264,265 are out of the five boroughs. Taxi zone 1 is the airport which are also out of these five boroughs. Thus, we delete these taxi zones.

In [5]:
def deal(r):
    if(r<103):
        return r-1
    else:
        return r-3

data_taxi=data_taxi[(data_taxi['PULocationID']!=264) & (data_taxi['PULocationID']!=265) & (data_taxi['PULocationID']!=1) &
          (data_taxi['DOLocationID']!=264) & (data_taxi['DOLocationID']!=265) & (data_taxi['DOLocationID']!=1)]

data_taxi['PULocationID']=data_taxi['PULocationID'].apply(lambda r:deal(r))
data_taxi['DOLocationID']=data_taxi['DOLocationID'].apply(lambda r:deal(r))
data_taxi=data_taxi.reset_index(drop=True)
data_taxi

,PULocationID,DOLocationID,num
0,55,170,1
1,170,170,1
2,55,170,1
3,55,126,1
4,55,55,1
...,...,...,...
16395289,134,86,1
16395290,134,70,1
16395291,134,31,1
16395292,134,156,1


In [6]:
data_zone=data_zone[(data_zone['LocationID']!=1) & (data_zone['LocationID']!=103) & (data_zone['LocationID']!=104) &
                    (data_zone['LocationID']!=264) & (data_zone['LocationID']!=265)]
data_zone=data_zone.reset_index(drop=True)
data_zone['LocationID']=data_zone['LocationID'].apply(lambda r:deal(r))
data_zone

,LocationID,Borough,Zone,service_zone
0,1,Queens,Jamaica Bay,Boro Zone
1,2,Bronx,Allerton/Pelham Gardens,Boro Zone
2,3,Manhattan,Alphabet City,Yellow Zone
3,4,Staten Island,Arden Heights,Boro Zone
4,5,Staten Island,Arrochar/Fort Wadsworth,Boro Zone
...,...,...,...,...
255,256,Bronx,Woodlawn/Wakefield,Boro Zone
256,257,Queens,Woodside,Boro Zone
257,258,Manhattan,World Trade Center,Yellow Zone
258,259,Manhattan,Yorkville East,Yellow Zone


In [7]:
data_taxi.to_csv('data_taxi.csv',index=False)
data_zone.to_csv('data_zone.csv',index=False)

# Calculate the steady vector and save

In [8]:
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import copy
import scipy.integrate as spi
import matplotlib.pyplot as plt
import pandas as pd
import geopandas as gpd
import gc

plt.rcParams['font.sans-serif']=['SimHei'] 
plt.rcParams['axes.unicode_minus']=False 

In [9]:
def deal(r):
    if(r<103):
        return r-1
    else:
        return r-3

# 加载出租车数据
data_taxi_fhv=pd.read_csv(r'C:\jupyter notebook\PAPER\data for taxi\fhv_tripdata_2020-03.csv')
data_taxi_fhvhv=pd.read_csv(r'C:\jupyter notebook\PAPER\data for taxi\fhvhv_tripdata_2020-03.csv')
data_taxi_green=pd.read_csv(r'C:\jupyter notebook\PAPER\data for taxi\green_tripdata_2020-03.csv')
data_taxi_yellow=pd.read_csv(r'C:\jupyter notebook\PAPER\data for taxi\yellow_tripdata_2020-03.csv')

data_taxi=pd.concat((data_taxi_fhv[['PULocationID','DOLocationID']],data_taxi_fhvhv[['PULocationID','DOLocationID']],data_taxi_green[['PULocationID','DOLocationID']],data_taxi_yellow[['PULocationID','DOLocationID']]),axis=0)
data_taxi=data_taxi.dropna().astype(int)
data_taxi['num']=1
data_taxi=data_taxi.reset_index(drop=True)

data_taxi=data_taxi[(data_taxi['PULocationID']!=264) & (data_taxi['PULocationID']!=265) & (data_taxi['PULocationID']!=1) &
          (data_taxi['DOLocationID']!=264) & (data_taxi['DOLocationID']!=265) & (data_taxi['DOLocationID']!=1)]

data_taxi['PULocationID']=data_taxi['PULocationID'].apply(lambda r:deal(r))
data_taxi['DOLocationID']=data_taxi['DOLocationID'].apply(lambda r:deal(r))
data_taxi=data_taxi.reset_index(drop=True)
data_taxi

,PULocationID,DOLocationID,num
0,55,170,1
1,170,170,1
2,55,170,1
3,55,126,1
4,55,55,1
...,...,...,...
16395289,134,86,1
16395290,134,70,1
16395291,134,31,1
16395292,134,156,1


In [10]:
data_OD1=data_taxi.groupby(['PULocationID','DOLocationID']).sum().unstack().fillna(0).astype(int)
data_OD1=data_OD1.applymap(lambda x:x+1)   
data_OD1=np.array(data_OD1).tolist()  


M=260 # the total number of taxi zones

# matrix of transition probability
P=copy.deepcopy(data_OD1)
for i in range(0,len(data_OD1)):
    for j in range(0,len(data_OD1)):
        P[i][j]=data_OD1[i][j]/sum(data_OD1[i])


# Steady state distribution vector
A=copy.deepcopy(P)
for i in range(0,M):
    for j in range(0,M):
        if(i==j):
            A[i][j]=1-A[i][j]
        else:
            A[i][j]=-A[i][j]

def get_reset(array):
    L=[]
    for i in range(len(array)):
        array_copy=copy.deepcopy(array)
        for x in range(len(array_copy)):
            array_copy[i][x]=0
        for y in range(len(array_copy)):
            array_copy[y][i]=0
        array_copy[i][i]=1
        L.append(np.linalg.det(array_copy))
    return L
KU=get_reset(A)
PI1=KU/sum(KU)

df=pd.DataFrame(PI1,columns=['202003'])
df=df.reset_index()
df.to_csv('202003.csv')
df

,index,202003
0,0,0.000020
1,1,0.002398
2,2,0.002976
3,3,0.000287
4,4,0.000569
...,...,...
255,255,0.002752
256,256,0.004019
257,257,0.002790
258,258,0.005272
